# 🎮 체스 AI 강화학습 (A2C)

**Pre-trained CNN 모델을 A2C 알고리즘과 Self-play로 강화학습합니다.**

## 학습 파이프라인
1. 지도학습 (train_cnn.ipynb) → Pre-trained 모델
2. **강화학습 (이 노트북)** → RL 모델
3. 평가 (eval_vs_stockfish.ipynb)

## 1. 환경 설정

In [1]:
import os
import copy
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter

# 프로젝트 모듈
from chess_model import ChessCNN, load_model
from train_utils import (
    compute_a2c_loss, train_step, play_games_batch, 
    evaluate_vs_opponent, check_weight_diff,
    compute_kl_divergence, compute_masked_entropy
)
import fast_chess as fc

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️ 디바이스: {device}")

# CUDA 메모리 최적화
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

🖥️ 디바이스: cuda
📊 GPU: NVIDIA GeForce RTX 5060 Ti
💾 VRAM: 17.1 GB


## 2. 하이퍼파라미터 설정

In [2]:
# =============================================================================
# 학습 설정
# =============================================================================
LEARNING_RATE = 1e-4          # Fine-tuning용 낮은 학습률
NUM_ITERATIONS = 10000        # 총 학습 반복 횟수
GAMES_PER_ITERATION = 12      # 반복당 Self-play 게임 수
NUM_ENVS = 12                 # 동시 진행 게임 수 (배치 크기)
MAX_MOVES = 200               # 게임당 최대 수

# =============================================================================
# 손실 함수 가중치
# =============================================================================
VALUE_LOSS_WEIGHT = 0.5       # Value Loss 가중치
ENTROPY_BONUS = 0.01          # 엔트로피 보너스 (탐색 장려)
KL_PENALTY = 0.1              # KL Divergence 패널티 초기값

# =============================================================================
# KL Divergence 설정 (Adaptive)
# =============================================================================
USE_ADAPTIVE_KL = True        # Adaptive KL 패널티 사용
KL_TARGET = 0.001             # 목표 KL 값
KL_PENALTY_MIN = 1e-4         # 최소 β 값
KL_PENALTY_MAX = 10.0         # 최대 β 값

# =============================================================================
# Temperature 스케줄
# =============================================================================
TEMPERATURE_START = 1.0       # 시작 탐색 온도
TEMPERATURE_END = 0.5         # 종료 탐색 온도

# =============================================================================
# 평가 및 저장
# =============================================================================
EVAL_INTERVAL = 10            # 평가 주기 (iteration)
EVAL_GAMES = 10               # 평가용 대결 게임 수
WIN_RATE_THRESHOLD = 0.5      # 저장 기준 승률 (50%+)
SAVE_INTERVAL = 50            # 체크포인트 저장 주기

# =============================================================================
# 경로 설정
# =============================================================================
PRETRAINED_PATH = 'models/best_chess_cnn.pth'
CHECKPOINT_PATH = 'models/chess_cnn_rl_a2c.pth'
BEST_MODEL_PATH = 'models/best_chess_cnn_rl_a2c.pth'
TENSORBOARD_DIR = 'models/tensorboard_rl'

print("✅ 하이퍼파라미터 설정 완료")
print(f"   - 학습률: {LEARNING_RATE}")
print(f"   - 반복 횟수: {NUM_ITERATIONS}")
print(f"   - 게임/반복: {GAMES_PER_ITERATION}")
print(f"   - 동시 게임: {NUM_ENVS}")

✅ 하이퍼파라미터 설정 완료
   - 학습률: 0.0001
   - 반복 횟수: 10000
   - 게임/반복: 12
   - 동시 게임: 12


## 3. 모델 로드

Pre-trained 모델을 로드하고, Reference 모델(고정)과 학습 모델을 준비합니다.

In [3]:
# Pre-trained 모델 로드
print("📥 Pre-trained 모델 로드 중...")

if os.path.exists(PRETRAINED_PATH):
    model, checkpoint = load_model(PRETRAINED_PATH, device)
    print(f"   ✅ {PRETRAINED_PATH} 로드 완료")
else:
    print(f"   ⚠️ {PRETRAINED_PATH} 없음 - 새 모델 생성")
    model = ChessCNN(num_channels=256).to(device)

# 학습 모델로 설정
model.train()

# Reference 모델 (Pre-trained 상태 고정)
print("🔒 Reference 모델 생성 중...")
ref_model = ChessCNN(num_channels=256).to(device)
ref_model.load_state_dict(model.state_dict())
ref_model.eval()

# Reference 모델 파라미터 동결
for param in ref_model.parameters():
    param.requires_grad = False

print("   ✅ Reference 모델 동결 완료")

# Best 모델 (평가용 상대)
best_model = ChessCNN(num_channels=256).to(device)
best_model.load_state_dict(model.state_dict())
best_model.eval()

print("   ✅ Best 모델 초기화 완료")

# 모델 파라미터 수
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 모델 파라미터:")
print(f"   - 전체: {total_params:,}")
print(f"   - 학습 가능: {trainable_params:,}")

   ✅ models/best_chess_cnn.pth 로드 완료
🔒 Reference 모델 생성 중...
   ✅ Reference 모델 동결 완료
   ✅ Best 모델 초기화 완료

📊 모델 파라미터:
   - 전체: 9,314,433
   - 학습 가능: 9,314,433


## 4. 옵티마이저 및 TensorBoard 설정

In [4]:
# 옵티마이저
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# AMP (Automatic Mixed Precision)
use_amp = torch.cuda.is_available()
scaler = torch.amp.GradScaler('cuda') if use_amp else None
print(f"⚡ AMP: {'활성화' if use_amp else '비활성화'}")

# TensorBoard
os.makedirs(TENSORBOARD_DIR, exist_ok=True)
writer = SummaryWriter(TENSORBOARD_DIR)
print(f"📊 TensorBoard: {TENSORBOARD_DIR}")

# 체크포인트에서 이어서 학습
start_iteration = 0
best_win_rate = 0.0

if os.path.exists(CHECKPOINT_PATH):
    ckpt = torch.load(CHECKPOINT_PATH, map_location=device, weights_only=False)
    if 'model_state_dict' in ckpt:
        model.load_state_dict(ckpt['model_state_dict'])
        if 'optimizer_state_dict' in ckpt:
            optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        start_iteration = ckpt.get('iteration', 0)
        best_win_rate = ckpt.get('best_win_rate', 0.0)
        KL_PENALTY = ckpt.get('kl_penalty', KL_PENALTY)
        print(f"✅ 체크포인트 로드: iteration {start_iteration}, win_rate {best_win_rate:.2%}")

print(f"\n🚀 학습 시작 준비 완료 (iteration {start_iteration}부터)")

⚡ AMP: 활성화
📊 TensorBoard: models/tensorboard_rl

🚀 학습 시작 준비 완료 (iteration 0부터)


## 5. 학습 루프

Self-play로 게임을 진행하고 A2C 알고리즘으로 학습합니다.

In [ ]:
from tqdm.auto import tqdm

print("🎮 강화학습 시작!")
print("=" * 60)

# 현재 KL Penalty (Adaptive용)
current_kl_penalty = KL_PENALTY

for iteration in tqdm(range(start_iteration, NUM_ITERATIONS), desc="학습 진행"):
    # =========================================================================
    # Temperature 스케줄 (선형 감소)
    # =========================================================================
    progress = iteration / max(NUM_ITERATIONS - 1, 1)
    temperature = TEMPERATURE_START + (TEMPERATURE_END - TEMPERATURE_START) * progress
    
    # =========================================================================
    # Self-play 게임 진행
    # =========================================================================
    trajectories, results, game_stats = play_games_batch(
        model=model,
        device=device,
        num_games=GAMES_PER_ITERATION,
        temperature=temperature,
        num_envs=NUM_ENVS,
        max_moves=MAX_MOVES
    )
    
    # =========================================================================
    # A2C 학습 스텝
    # =========================================================================
    train_metrics = train_step(
        model=model,
        ref_model=ref_model,
        optimizer=optimizer,
        trajectories=trajectories,
        results=results,
        device=device,
        value_loss_weight=VALUE_LOSS_WEIGHT,
        entropy_bonus=ENTROPY_BONUS,
        kl_penalty=current_kl_penalty,
        max_grad_norm=1.0,
        scaler=scaler,
        use_amp=use_amp
    )
    
    # =========================================================================
    # Adaptive KL Penalty
    # =========================================================================
    if USE_ADAPTIVE_KL:
        avg_kl = train_metrics['kl_div']
        if avg_kl > KL_TARGET * 1.5:
            current_kl_penalty = min(current_kl_penalty * 1.5, KL_PENALTY_MAX)
        elif avg_kl < KL_TARGET * 0.5:
            current_kl_penalty = max(current_kl_penalty / 1.5, KL_PENALTY_MIN)
    
    # =========================================================================
    # TensorBoard 로깅
    # =========================================================================
    writer.add_scalar('Loss/Total', train_metrics['total_loss'], iteration)
    writer.add_scalar('Loss/Policy', train_metrics['policy_loss'], iteration)
    writer.add_scalar('Loss/Value', train_metrics['value_loss'], iteration)
    writer.add_scalar('Entropy', train_metrics['entropy'], iteration)
    writer.add_scalar('KL_Divergence', train_metrics['kl_div'], iteration)
    writer.add_scalar('KL_Penalty', current_kl_penalty, iteration)
    writer.add_scalar('Temperature', temperature, iteration)
    writer.add_scalar('Games/Avg_Moves', game_stats['avg_moves'], iteration)
    writer.add_scalar('Games/White_Wins', game_stats['white_wins'], iteration)
    writer.add_scalar('Games/Black_Wins', game_stats['black_wins'], iteration)
    writer.add_scalar('Games/Draws', game_stats['draws'], iteration)
    
    # =========================================================================
    # 평가 (주기적)
    # =========================================================================
    if (iteration + 1) % EVAL_INTERVAL == 0:
        win_rate = evaluate_vs_opponent(
            current_model=model,
            opponent_model=best_model,
            device=device,
            num_games=EVAL_GAMES,
            temperature=0.5
        )
        
        writer.add_scalar('Win_Rate', win_rate, iteration)
        
        # 최고 모델 갱신
        if win_rate >= WIN_RATE_THRESHOLD and win_rate > best_win_rate:
            best_win_rate = win_rate
            best_model.load_state_dict(model.state_dict())
            
            # Best 모델 저장
            torch.save({
                'model_state_dict': model.state_dict(),
                'iteration': iteration,
                'win_rate': win_rate,
                'kl_penalty': current_kl_penalty,
            }, BEST_MODEL_PATH)
            
            tqdm.write(f"🏆 [Iter {iteration+1}] 새로운 최고 모델! Win Rate: {win_rate:.2%}")
        else:
            tqdm.write(f"📊 [Iter {iteration+1}] Win Rate: {win_rate:.2%} (Best: {best_win_rate:.2%})")
    
    # =========================================================================
    # 체크포인트 저장 (주기적)
    # =========================================================================
    if (iteration + 1) % SAVE_INTERVAL == 0:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'iteration': iteration + 1,
            'best_win_rate': best_win_rate,
            'kl_penalty': current_kl_penalty,
        }, CHECKPOINT_PATH)
        tqdm.write(f"💾 [Iter {iteration+1}] 체크포인트 저장")
    
    # =========================================================================
    # 진행 상황 출력 (100 iteration마다)
    # =========================================================================
    if (iteration + 1) % 10 == 0:
        weight_diff = check_weight_diff(model, ref_model)
        tqdm.write(f"📈 [Iter {iteration+1}] Loss: {train_metrics['total_loss']:.4f}, "
                   f"Entropy: {train_metrics['entropy']:.4f}, "
                   f"KL: {train_metrics['kl_div']:.6f}, "
                   f"Moves: {game_stats['avg_moves']:.1f}, "
                   f"Weight Diff: {weight_diff:.4f}")

print("\n" + "=" * 60)
print("✅ 학습 완료!")

# 최종 저장
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'iteration': NUM_ITERATIONS,
    'best_win_rate': best_win_rate,
    'kl_penalty': current_kl_penalty,
}, CHECKPOINT_PATH)
print(f"💾 최종 체크포인트 저장: {CHECKPOINT_PATH}")

writer.close()

학습 진행:   0%|          | 23/10000 [03:00<21:05:13,  7.61s/it]

## 6. 학습 결과 분석

In [ ]:
# 가중치 변화 확인
weight_diff = check_weight_diff(model, ref_model)
print(f"📊 학습 결과 분석")
print("=" * 60)
print(f"   - Reference 모델과의 가중치 차이: {weight_diff:.6f}")

if weight_diff < 1e-6:
    print("   ⚠️ 경고: 모델 가중치가 거의 변하지 않았습니다!")
    print("      → 학습률이 너무 낮거나, KL 패널티가 너무 높을 수 있습니다.")
else:
    print("   ✅ 모델이 정상적으로 학습되었습니다.")

print(f"\n   - 최고 승률: {best_win_rate:.2%}")
print(f"   - 최종 KL 패널티: {current_kl_penalty:.6f}")

# 최종 평가
print("\n🎯 최종 평가 (vs Best Model)...")
final_win_rate = evaluate_vs_opponent(
    current_model=model,
    opponent_model=best_model,
    device=device,
    num_games=20,
    temperature=0.3
)
print(f"   - 승률: {final_win_rate:.2%}")

## 7. 샘플 게임 시각화

학습된 모델로 샘플 게임을 진행합니다.

In [ ]:
def play_demo_game(model, device, temperature=0.3, max_moves=100, verbose=True):
    """샘플 게임을 진행하고 결과를 출력합니다."""
    import fast_chess as fc
    
    state = fc.create_initial_state()
    model.eval()
    
    moves_history = []
    
    for move_num in range(max_moves):
        if fc.is_game_over(state):
            break
        
        state_tensor = torch.from_numpy(fc.board_to_tensor_fast(state))
        mask_tensor = torch.from_numpy(fc.legal_move_mask_fast(state))
        
        if mask_tensor.sum() == 0:
            break
        
        with torch.no_grad():
            state_input = state_tensor.unsqueeze(0).to(device)
            mask_input = mask_tensor.unsqueeze(0).to(device)
            policy_logits, value_pred = model(state_input, mask_input)
        
        # Greedy 선택 (temperature=0.3)
        from train_utils import select_action
        action, _, _ = select_action(policy_logits.squeeze(0), mask_tensor.to(device), temperature)
        
        # 수 변환
        from_sq = action // 64
        to_sq = action % 64
        files = "abcdefgh"
        ranks = "12345678"
        move_str = f"{files[from_sq & 7]}{ranks[from_sq >> 3]}{files[to_sq & 7]}{ranks[to_sq >> 3]}"
        moves_history.append(move_str)
        
        fc.make_move(state, action)
    
    # 결과
    result = fc.get_result(state)
    if result == 1.0:
        result_str = "1-0 (백 승)"
    elif result == 0.0:
        result_str = "0-1 (흑 승)"
    else:
        result_str = "1/2-1/2 (무승부)"
    
    if verbose:
        print(f"🎮 샘플 게임 결과: {result_str}")
        print(f"   총 {len(moves_history)}수")
        print(f"   수순: {' '.join(moves_history[:20])}{'...' if len(moves_history) > 20 else ''}")
    
    return moves_history, result_str

# 샘플 게임 3개 진행
print("🎲 샘플 게임 진행 중...")
print("=" * 60)

for i in range(3):
    print(f"\n게임 #{i+1}")
    play_demo_game(model, device, temperature=0.3)